**Import librairies**

In [ ]:
%pip install pytorch-lightning==1.9.4

In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import CelebA
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_lightning import LightningModule
from torch.nn import Conv2d, ConvTranspose2d, Flatten, Linear, LeakyReLU, Dropout, MaxPool2d, MSELoss, BatchNorm2d, ReLU, Tanh
import torch.nn as nn
import torch.optim as optim
import random
import torchvision.utils as vutils

In [ ]:
# Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True)

Random Seed:  999


**Load the dataset**

Real-world images are taken from the [CelebA dataset](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)

In [ ]:
!unzip /content/drive/MyDrive/img_align_celeba.zip

Archive:  /content/drive/MyDrive/img_align_celeba.zip
replace img_align_celeba/000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
class CelebADataset(Dataset):
  def __init__(self, root_dir):
    self.img_names = os.listdir(root_dir)
    self.root_dir = root_dir
    self.transform = transforms.Compose([transforms.Resize(64),
                                        transforms.CenterCrop(64),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  def __len__(self):
    return len(self.img_names)

  def __getitem__(self, idx):
    img_path = os.path.join(self.root_dir, self.img_names[idx])
    img = Image.open(img_path).convert('RGB')
    img = self.transform(img)
    return img

In [ ]:
dataset = CelebADataset('/content/img_align_celeba')

In [ ]:
print(len(dataset))

In [ ]:
img0 = dataset[0]
print(img0.shape)
plt.imshow(img0.permute(1, 2, 0))

**Generator**

The task of the Generator is to create a fake image which is indistinguishable from a real image

In [ ]:
# GENERATOR ARCHITECTURE

class Generator(nn.Module):

    def __init__(self):
        super().__init__()
        self.convT1 = ConvTranspose2d(in_channels=100, out_channels=512,
                                      kernel_size=4, stride=1, padding=0)
        self.norm1 = BatchNorm2d(512)
        self.convT2 = ConvTranspose2d(in_channels=512, out_channels=256,
                                      kernel_size=4, stride=2, padding=1)
        self.norm2 = BatchNorm2d(256)
        self.convT3 = ConvTranspose2d(in_channels=256, out_channels=128,
                                      kernel_size=4, stride=2, padding=1)
        self.norm3 = BatchNorm2d(128)
        self.convT4 = ConvTranspose2d(in_channels=128, out_channels=64,
                                      kernel_size=4, stride=2, padding=1)
        self.norm4 = BatchNorm2d(64)
        self.convT5 = ConvTranspose2d(in_channels=64, out_channels=3,
                                      kernel_size=4, stride=2, padding=1)
        self.relu = ReLU(inplace=True)
        self.tanh = Tanh()

    def forward(self, x):
        x1 = self.relu(self.norm1(self.convT1(x)))
        x2 = self.relu(self.norm2(self.convT2(x1)))
        x3 = self.relu(self.norm3(self.convT3(x2)))
        x4 = self.relu(self.norm4(self.convT4(x3)))
        x5 = self.tanh(self.convT5(x4))
        return x5

In [ ]:
gen = Generator()

**Discriminator**

The task of the Discriminator is to distinguish between 2 cases - real (from the domain) and fake images (generated).

In [ ]:
# DISCRIMINATOR ARCHITECTURE

class Discriminator(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=64, kernel_size=4,
                            stride=2, padding=1, bias=False)
        self.conv2 = Conv2d(in_channels=64, out_channels=128, kernel_size=4,
                            stride=2, padding=1, bias=False)
        self.norm2 = BatchNorm2d(128)
        self.conv3 = Conv2d(in_channels=128, out_channels=256, kernel_size=4,
                            stride=2, padding=1, bias=False)
        self.norm3 = BatchNorm2d(256)
        self.conv4 = Conv2d(in_channels=256, out_channels=512, kernel_size=4,
                            stride=2, padding=1, bias=False)
        self.norm4 = BatchNorm2d(512)
        self.conv5 = Conv2d(in_channels=512, out_channels=1, kernel_size=4,
                            stride=1, padding=0, bias=False)
        self.l_relu = LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        x1 = self.l_relu(self.conv1(x))
        x2 = self.l_relu(self.norm2(self.conv2(x1)))
        x3 = self.l_relu(self.norm3(self.conv3(x2)))
        x4 = self.l_relu(self.norm4(self.conv4(x3)))
        pred = torch.sigmoid(self.conv5(x4))
        return pred

In [ ]:
disc = Discriminator()

**GAN Training**

In [ ]:
class Trainer():
  def __init__(self, generator, discriminator):
      self.generator = generator
      self.generator.train()
      self.discriminator = discriminator
      self.discriminator.train()

      # Loss function is Binary Cross Entropy between the target and the input probabilities
      self.loss_fct = nn.BCELoss()

      # Instantiate optimizer for the generator
      self.generator_optim = optim.Adam(self.generator.parameters(),
                                      lr=0.002,
                                      betas=(0.5, 0.999))
      # Instantiate optimizer for the discriminator
      self.discriminator_optim = optim.Adam(self.discriminator.parameters(),
                                          lr=0.002,
                                          betas=(0.5, 0.999))

  def train(self, train_dataset, epochs):
      batch_counter = 0
      batch_size = 50
      latent_dim = 100 # for noise
      disc_losses = []
      gen_losses = []

      for epoch in range(epochs):

        print("")
        print("epoch", epoch)

        self.discriminator_optim.zero_grad()
        # Labels generated are soft labels:
        # rather than being 1, they take values between 0.7 and 1.2
        real_labels = (1.2 - 0.7) * torch.rand((batch_size,)) + 0.7
        # rather than being 0, they take values between 0.3 and 0.0
        fake_labels = (0.3 - 0.0) * torch.rand((batch_size,)) + 0.0
        real_imgs = torch.empty((batch_size, 3, 64, 64))

        # Update the Discriminator network:
        # maximize log(D(x)) + log(1 - D(G(z)))
        for i in range(batch_size):
          j = i + batch_counter
          # Train with the batch of real images
          real_imgs[i] = train_dataset[j].unsqueeze(0)
        # Forward pass the batch of real images through the discriminator
        output = self.discriminator(real_imgs).squeeze(3).squeeze(2).squeeze(1)
        batch_counter += batch_size
        # Calculate loss on real batch
        real_discr_loss = self.loss_fct(output, real_labels)
        # Calculate gradients for the real batch
        real_discr_loss.backward(retain_graph=True)
        # Get the D(x)
        D_x = output.mean().item()
        print("D(x): ", D_x)

        # Train both the discriminator and generator with the fake batch
        # Generate a batch of latent vectors
        noise = torch.randn((batch_size, latent_dim, 1, 1))
        # Generate fake images from noise
        fake = self.generator(noise)
        # Forward pass the batch of fake images through the discriminator
        output = self.discriminator(fake).squeeze(3).squeeze(2).squeeze(1)
        # Calculate loss on fake batch
        fake_discr_loss = self.loss_fct(output, fake_labels)
        # Calculate gradients for the fake batch
        fake_discr_loss.backward(retain_graph=True)
        # Get the D(G(z1))
        D_G_z1 = fake_discr_loss.mean().item()
        discr_loss = (real_discr_loss + fake_discr_loss)

        # Update weights for the discriminator
        self.discriminator_optim.step()

        # Save losses for plotting
        disc_losses.append(discr_loss.item())

        print("epoch discr loss: ", discr_loss.item())

        # Update the Generator network:
        # maximize log(D(G(z)))
        self.generator_optim.zero_grad()
        # Forward pass the batch of fake images through the discriminator
        output = self.discriminator(fake).squeeze(3).squeeze(2).squeeze(1)
        # Calculate the generator's loss : here fake labels are interchanged with real labels
        gen_loss = self.loss_fct(output, real_labels)
        # Calculate gradients for the generator
        gen_loss.backward(retain_graph=True)
        # Get the D(G(z2))
        D_G_z2 = output.mean().item()

        print("D(G(x)): ", D_G_z1, "/", D_G_z2)

        # Update the weights for the generator
        self.generator_optim.step()

        # Save losses for plotting
        gen_losses.append(gen_loss.item())

        print("epoch gen loss: ", gen_loss.item())

      return disc_losses, gen_losses

  def evaluate(self, test_dataset):
        right_pred = 0
        counter = 0
        with torch.no_grad():
            for i in range(6500, 7500):
                input_real = test_dataset[i].unsqueeze(0)
                output_real = self.discriminator(input_real)
                if output_real.item() >= 0.5 : right_pred += 1

                test_noise = torch.randn((1, 100, 1, 1))
                input_fake = self.generator(test_noise)
                output_fake = self.discriminator(input_fake)
                if output_fake.item() < 0.5 : right_pred += 1
        return right_pred/2000

In [ ]:
trainer = Trainer(generator=gen,
                  discriminator=disc)

In [ ]:
nb_epochs = 250
losses = trainer.train(dataset, nb_epochs)

In [ ]:
epoch_count = range(0, nb_epochs)
D_losses = losses[0]
plt.title('Evolution of the Discriminator loss')
plt.plot(epoch_count, D_losses)

In [ ]:
epoch_count = range(0, nb_epochs)
G_losses = losses[1]
plt.title('Evolution of the Generator loss')
plt.plot(epoch_count, G_losses)

Testing

In [ ]:
avg_test_loss = trainer.evaluate(dataset)
print("Average testing loss: ", avg_test_loss)

Saving model

In [ ]:
torch.save(disc.state_dict(), "discriminator_260.pth")
torch.save(gen.state_dict(), "generator_260.pth")

In [ ]:
one_latent = torch.rand((1, 100, 1, 1))
rd0 = gen(one_latent).squeeze(0)
print(rd0.shape)
plt.imshow(rd0.permute(1, 2, 0).detach().numpy())